In [4]:
# !pip install psycopg2
# !pip install pandas 

# you first have to install these packages if you dont already have them, I already have them so i wont install again

import psycopg2 # to connect to our database server
import pandas as pd # to read, manipulate, upload the data

In [3]:
# now we will create a function to create our database
def create_database():
    try:
        conn = psycopg2.connect("host = localhost dbname = postgres user = postgres password = root150") #connecting to the default database
    except psycopg2.Error as e :
        print("error, couldnt connect to database")   
        print(e)
    conn.set_session(autocommit = True) # any changes we make from here will be updated automatically in the database
    curr = conn.cursor()
    
    curr.execute("Create Database airbnb")   # creating our database in postgres
    conn.close()   # closing connection to the default database
    
    try:
        conn = psycopg2.connect("host = localhost dbname = airbnb user = postgres password = root150") # connecting to our new database
    except psycopg2.Error as e :
        print("error, couldnt connect to database")
        print(e)
    conn.set_session(autocommit = True)
    curr = conn.cursor() # creating a cursor to be able to communicate to our database
    
    return curr, conn

In [ ]:
# in the next section, we will read all the files and filter all the data that we require

In [5]:
review_details = pd.read_csv("/Users/ADMIN/Desktop/airbnbdf/reviews_details.csv") # reading the csv file

In [6]:
review_details.head() # displaying first 5 rows

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [7]:
reviews = review_details[['id', 'comments']] # copying the desired columns into a separate data frame

In [8]:
listings_raw = pd.read_csv("/Users/ADMIN/Desktop/airbnbdf/listings.csv") # reading the csv file

In [9]:
listings_raw.head() # displaying first 5 rows

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.365755,4.941419,Private room,59,3,248,2018-11-28,2.10,1,44
1,3209,"Quiet apt near center, great view",3806,Maartje,NaN,Westerpark,52.390225,4.873924,Entire home/apt,160,4,42,2018-08-29,1.03,1,47
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,59484,Alex,NaN,Centrum-Oost,52.365087,4.893541,Entire home/apt,80,1,233,2018-11-30,2.18,2,198
3,25428,Lovely apt in City Centre (Jordaan),56142,Joan,NaN,Centrum-West,52.373114,4.883668,Entire home/apt,125,14,1,2018-01-21,0.09,2,141
4,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.386727,4.892078,Private room,150,2,171,2018-11-25,2.03,1,199


In [10]:
listings = listings_raw[['id', 'name', 'room_type', 'price']] # copying the desired columns into a separate data frame

In [15]:
listing_details_raw = pd.read_csv("/Users/ADMIN/Desktop/airbnbdf/listings_details.csv", low_memory = False) # reading the csv file

In [16]:
listing_details_raw.head() # displaying first 5 rows

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20181206172549,2018-12-06,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.10
1,3209,https://www.airbnb.com/rooms/3209,20181206172549,2018-12-06,"Quiet apt near center, great view",You will love our spacious (90 m2) bright apar...,"Our apartment has lots of light, a balcony and...",You will love our spacious (90 m2) bright apar...,none,Welcome to the Spaarndammerbuurt! From the beg...,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,1.03
2,20168,https://www.airbnb.com/rooms/20168,20181206172549,2018-12-06,100%Centre-Studio 1 Private Floor/Bathroom,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,2.18
3,25428,https://www.airbnb.com/rooms/25428,20181206172549,2018-12-06,Lovely apt in City Centre (Jordaan),NaN,"This nicely furnished, newly renovated apt is...","This nicely furnished, newly renovated apt is...",none,NaN,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,0.09
4,27886,https://www.airbnb.com/rooms/27886,20181206172549,2018-12-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,For a romantic couple: A beautifully restored ...,Stylish and romantic houseboat on fantastic hi...,none,"Central, quiet, safe, clean and beautiful.",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.03


In [17]:
listing_details = listing_details_raw[['id', 'last_scraped', 'summary', 'description']] # copying the desired columns into a separate data frame

In [18]:
curr, conn = create_database() # calling the function to create the database

In [34]:
reviews_create = ("""Create table if not exists reviews(
id int Primary key, 
comments varchar)""") # storing our SQL query to create a table inside a variable

In [35]:
curr.execute(reviews_create) # Executing our SQL query

In [23]:
listings_create = ("""Create table if not exists listings(
id int,
name varchar,
room_type varchar,
price int)""")  

In [24]:
curr.execute(listings_create) 

In [25]:
listing_details_create = ("""create table if not exists listing_details(
id int, 
last_scraped date,
summary varchar,
description varchar)""") 

In [26]:
curr.execute(listing_details_create) 

In [36]:
reviews_insert = ("""insert into reviews(
id,
comments)
values (%s, %s)""") # storing a SQL query to insert data into a table inside a variable

In [37]:
for i, row in reviews.iterrows(): # iterrows returns index and information stored in a row of a data frame
    curr.execute(reviews_insert, list(row)) # So we are reading each row from reviews dataframe and inserting it into reviews
                                            # table 

In [30]:
listings_insert = ("""insert into listings(
id,
name,
room_type,
price)
values (%s, %s, %s, %s)""") 

In [31]:
for i, row in listings.iterrows():
    curr.execute(listings_insert, list(row))

In [32]:
listing_details_insert = ("""insert into listing_details(
id, 
last_scraped,
summary,
description)
values (%s, %s, %s, %s)
""")

In [33]:
for i, row in listing_details.iterrows():
    curr.execute(listing_details_insert, list(row))